In [ ]:
from cellpose import models, io, core
import os
import glob
from PIL import Image
import shutil
import time

# Text styling
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

# Check if GPU is being used
if not core.use_gpu(): 
  print(BOLD+UNDERLINE+'You do not have GPU access.'+END)
  use_GPU=False
else:
  print(BOLD+UNDERLINE+"You have access to the GPU."+END+"\nDetails are:")
  print("*************************************************")
  !nvidia-smi
  use_GPU=True

print("*************************************************")
print("Libraries imported and configured")

In [6]:
# Filepath to images:
inputDir = r"G:\Shared\lab_members\undergraduate_researchers\yung_maxx\Experiments\Cellpose 05-01-2024\Run 1 - Custom Model"

# Model Setup:
modelChoice = "cyto2" # nuclei, cyto2

customModel = True # True if you want to use a custom model & then set the path below. Otherwise, "False". If true, it will use the below model over the above modelChoice parameter.
customModelPath = r"C:\Users\myung11\.cellpose\models\model_FITC"
imageSuffix = "Any" # DAPI, CY5, Any

# Misc Setup
cellDiameter = None # if None, diameter of cells is automatically estimated
cellFlowThreshold = 2 # threshold for cell probability. Increase if you expect more ROIs.
cellProbabilityThreshold = -0.5 # threshold for cell probability. Increase this threshold if cellpose is returning too ROIs particularly from dim areas.

# Segmentation Settings
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0] IF YOU HAVE GRAYSCALE
# channels = [2,3] IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] IF YOU HAVE G=cytoplasm and R=nucleus

channels = [0,0]

In [7]:
os.chdir(inputDir)
files = glob.glob('*.tif')
print(str(len(files)) + " total files found in directory.")
outputDir = inputDir

if os.path.exists(outputDir + "/ImageJ Results/"):
    shutil.rmtree(outputDir + "/ImageJ Results/")
os.makedirs(outputDir + "/ImageJ Results/")

# Filters
def delete_non_tif(directory):
    for filename in glob.glob(os.path.join(directory, '*')):
        if os.path.isdir(filename):
            delete_non_tif(filename)
        elif not filename.endswith('.tif'):
            os.remove(filename)

delete_non_tif(outputDir)

if imageSuffix == "DAPI":
    files = glob.glob(os.path.join(outputDir, "*_DAPI.tif"))
    print(str(len(files)) + " DAPI files will be processed.")
elif imageSuffix == "CY5":
    files = glob.glob(os.path.join(outputDir, "*_CY5.tif"))
    print(str(len(files)) + " CY5 files will be processed.")
else: 
    files = files
    print("All " + str(len(files)) + " files will be processed.")
    
print("Non-TIFF files removed from output directory.")
print("\nResults will be saved in: " + outputDir)

# Model Determination
if customModel is True:
	model = models.CellposeModel(gpu=use_GPU, pretrained_model=customModelPath)
	print("Using custom model from: " + customModelPath)
else:
	model = models.Cellpose(gpu=use_GPU, model_type=modelChoice)
	print("Using default model: " + modelChoice)
  
if cellDiameter is None:
    cellDiameter = 0
    print("\nCell diameter not specified (set to 0). Automatically estimating cell diameter.")
else:
    print("\nCell diameter: " + str(cellDiameter))
  
print("Flow threshold: " + str(cellFlowThreshold) + "\nProbability threshold: " + str(cellProbabilityThreshold) + "\n")

# Pixel Count
max_pixel_count = 9000000
pixelCount = 0

for file in files:
    with Image.open(file) as img:
        width, height = img.size
        pixel_count = width * height
        pixelCount += pixel_count

        if pixel_count > max_pixel_count:
            print(f"WARNING: Image {file} has too many pixels, segmentation might take longer. Please consider resizing the image to a smaller size.")
            
print("\nSegmentation will begin now. Estimated time for completion is " + str(pixelCount//16000000) + " minutes.\n")

start_time = time.time()

# Processing
for i in files:
    image = io.imread(i)
    print("Processing: " + i)
    if customModel is True:
        masks, flows, styles = model.eval(image, diameter=cellDiameter, flow_threshold=cellFlowThreshold,cellprob_threshold=cellProbabilityThreshold, channels=channels)
    else:
        masks, flows, styles, diams = model.eval(image, diameter=cellDiameter, flow_threshold=cellFlowThreshold,cellprob_threshold=cellProbabilityThreshold, channels=channels)
    outputName = os.path.join(outputDir, i)
    io.save_rois(masks, outputName)
    io.masks_flows_to_seg(image, masks, flows, cellDiameter, outputName, channels=channels)
    print("Processed: " + i)

end_time = time.time()
 
#Save parameters used in Cellpose
parametersDir = os.path.join(outputDir, "Cellpose_Parameters.txt") 
outFile=open(parametersDir, "w") 
outFile.write("CELLPOSE PARAMETERS\n")
if customModel is True:
    outFile.write("Custom Model: "+ customModelPath + "\n") 
else:
    outFile.write("Default Model: "+ modelChoice + "\n") 
if cellDiameter == 0:
    outFile.write("Cell Diameter was estimated since cellDiameter = 0")
else:
    outFile.write("Diameter: " + str(cellDiameter) + "\n") 

outFile.write("\nFlow Threshold: " + str(cellFlowThreshold)) 
outFile.write("\nCell Probability Threshold: " + str(cellProbabilityThreshold) + "\n") 
outFile.close()

print(f"\nThe program took {round((end_time-start_time)/60, 1)} minutes to run.")
print("Segmentation complete and files saved")
os.startfile(outputDir)

30 total files found in directory.
All 30 files will be processed.
Non-TIFF files removed from output directory.

Results will be saved in: G:\Shared\lab_members\undergraduate_researchers\yung_maxx\Experiments\Cellpose 05-01-2024\Run 1 - Custom Model
Using custom model from: C:\Users\myung11\.cellpose\models\model_FITC

Cell diameter not specified (set to 0). Automatically estimating cell diameter.
Flow threshold: 2
Probability threshold: -0.5


Segmentation will begin now. Estimated time for completion is 39 minutes.

Processing: Mouse 2111.643_Slide3_Slice10L_20X_FITC.tif


Multiprocessing is disabled for Windows


Processed: Mouse 2111.643_Slide3_Slice10L_20X_FITC.tif
Processing: Mouse 2111.643_Slide3_Slice1L_20X_FITC.tif


Multiprocessing is disabled for Windows


Processed: Mouse 2111.643_Slide3_Slice1L_20X_FITC.tif
Processing: Mouse 2111.643_Slide3_Slice2L_20X_FITC.tif


...

Processed: Mouse 2111.643_Slide3_Slice2L_20X_FITC.tif
Processing: Mouse 2111.643_Slide3_Slice4L_20X_FITC.tif


Multiprocessing is disabled for Windows


Processed: Mouse 2111.643_Slide3_Slice4L_20X_FITC.tif
Processing: Mouse 2111.643_Slide3_Slice5L_20X_FITC.tif


Multiprocessing is disabled for Windows


Processed: Mouse 2111.643_Slide3_Slice5L_20X_FITC.tif
Processing: Mouse 2111.643_Slide3_Slice7L_20X_FITC.tif
